In [12]:
%load_ext autoreload
%autoreload 2

import sys
import os

sys.path.append(os.path.join(os.getcwd(), ".."))

from models.sp500.fundamentals import FundamentalStockAnalyzer
import logging
import yfinance as yf
import time

logging.basicConfig(level=logging.CRITICAL)  # or DEBUG for more detail

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
# Create analyzer using the new async factory method
async def setup_analyzer():
    # Use the async factory method instead of direct constructor
    fas = await FundamentalStockAnalyzer.create()

    # Get tickers (now async)
    tickers = await fas.tickers
    print(f"Loaded {len(tickers)} S&P 500 tickers")
    print(f"First 10 tickers: {tickers[:10]}")

    # Show available field sets (this is still synchronous)
    print(f"\nAvailable field sets: {fas.get_available_field_sets()}")

    return fas


# Run the async setup
fas = await setup_analyzer()

Loaded 503 S&P 500 tickers
First 10 tickers: ['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ADBE', 'AMD', 'AES', 'AFL', 'A']

Available field sets: ['quick_overview', 'value_analysis', 'fundamental_analysis', 'price_performance', 'complete', 'basic_info', 'valuation_metrics', 'profitability_metrics', 'cash_flow_metrics', 'growth_metrics', 'percentage_changes', 'financial_health', 'dividend_info', 'analyst_data', 'earnings_data', 'all']


In [14]:
fas = await FundamentalStockAnalyzer.create()
tickers = await fas.tickers
print(f"Loaded {len(tickers)} S&P 500 tickers")
print(f"First 10 tickers: {tickers[:10]}")

# Show available field sets (this is still synchronous)
print(f"\nAvailable field sets: {fas.get_available_field_sets()}")

Loaded 503 S&P 500 tickers
First 10 tickers: ['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ADBE', 'AMD', 'AES', 'AFL', 'A']

Available field sets: ['quick_overview', 'value_analysis', 'fundamental_analysis', 'price_performance', 'complete', 'basic_info', 'valuation_metrics', 'profitability_metrics', 'cash_flow_metrics', 'growth_metrics', 'percentage_changes', 'financial_health', 'dividend_info', 'analyst_data', 'earnings_data', 'all']


In [15]:
# Test single ticker fundamental data (now async)
print("=== Single Ticker Example ===")
result = await fas.get_fundamentals("AAPL", field_set="basic_info")
print(f"AAPL basic info: {result}")

print("\n=== Error Handling Example ===")
error_result = await fas.get_fundamentals("INVALID_TICKER")
print(f"Invalid ticker result: {error_result}")

=== Single Ticker Example ===
AAPL basic info: {'ticker': 'AAPL', 'data': {'shortName': 'Apple Inc.', 'sector': 'Technology', 'industry': 'Consumer Electronics', 'marketCap': 3473692426240, 'currentPrice': 234.07, 'beta': 1.109}}

=== Error Handling Example ===
Invalid ticker result: {'ticker': 'INVALID_TICKER', 'data': {'dividendRate': None, 'oneWeekChangePercent': None, 'sixMonthChangePercent': None, 'targetMeanPrice': None, 'returnOnAssets': None, 'debtToEquity': None, 'sector': None, 'marketCap': None, 'trailingEps': None, 'forwardPE': None, 'totalCash': None, 'totalCashPerShare': None, 'industry': None, 'currentPrice': None, 'earningsGrowth': None, 'beta': None, 'operatingCashflow': None, 'threeMonthChangePercent': None, 'priceToBook': None, 'shortName': None, 'trailingAnnualDividendRate': None, 'enterpriseValue': None, 'ebitda': None, 'returnOnEquity': None, 'freeCashflow': None, 'recommendationKey': None, 'forwardEps': None, 'oneMonthChangePercent': None, 'annualChangePercent': 

In [16]:
# Test multiple tickers (small sample) - now with async and concurrency control
print("=== Multiple Tickers Example ===")
sample_tickers = ["AAPL", "MSFT", "GOOGL"]

start_time = time.time()

results = await fas.get_multiple_fundamentals(
    tickers=sample_tickers,
    field_set="basic_info",  # Using basic_info since value_analysis might not exist
    max_concurrent=3,  # Process all 3 concurrently
)

end_time = time.time()
print(f"Fetched {len(sample_tickers)} tickers in {end_time - start_time:.2f} seconds")

for result in results:
    if "error" not in result:
        ticker = result["ticker"]
        data = result["data"]
        short_name = data.get("shortName", "Unknown")
        market_cap = data.get("marketCap", "N/A")
        print(f"{ticker}: {short_name} - Market Cap: {market_cap}")
    else:
        print(f"{result['ticker']}: Error - {result['error']}")

print("\n=== Configuration Examples ===")
print(f"Cache days setting: {fas.config.cache_days}")
print(f"Data source URL: {fas.config.data_url}")

# Show different field sets
basic_fields = fas.get_fields("basic_info")
all_field_sets = fas.get_available_field_sets()
print(f"Basic info fields ({len(basic_fields)}): {basic_fields}")
print(f"All available field sets: {all_field_sets}")
if "value_analysis" in all_field_sets:
    value_fields = fas.get_fields("value_analysis")
    print(f"Value analysis fields ({len(value_fields)}): {value_fields}")

=== Multiple Tickers Example ===
Fetched 3 tickers in 0.33 seconds
AAPL: Apple Inc. - Market Cap: 3473692426240
MSFT: Microsoft Corporation - Market Cap: 3790173372416
GOOGL: Alphabet Inc. - Market Cap: 2915389669376

=== Configuration Examples ===
Cache days setting: 30
Data source URL: https://en.wikipedia.org/wiki/List_of_S%26P_500_companies
Basic info fields (6): ['shortName', 'sector', 'industry', 'marketCap', 'currentPrice', 'beta']
All available field sets: ['quick_overview', 'value_analysis', 'fundamental_analysis', 'price_performance', 'complete', 'basic_info', 'valuation_metrics', 'profitability_metrics', 'cash_flow_metrics', 'growth_metrics', 'percentage_changes', 'financial_health', 'dividend_info', 'analyst_data', 'earnings_data', 'all']
Value analysis fields (6): ['priceToBook', 'trailingPE', 'forwardPE', 'pegRatio', 'priceToSalesTrailing12Months', 'enterpriseToEbitda']


In [17]:
stock = yf.Ticker("V")
hist = stock.history(period="1y")
hist.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2024-09-13 00:00:00-04:00,284.492112,286.726421,283.558665,285.346130,4987600,0.0,0.0
2024-09-16 00:00:00-04:00,286.041238,288.871369,285.733402,288.454315,4458900,0.0,0.0
2024-09-17 00:00:00-04:00,288.871363,291.026243,288.057074,289.526764,5135700,0.0,0.0
2024-09-18 00:00:00-04:00,289.804816,289.983553,286.120696,286.468262,5890000,0.0,0.0
2024-09-19 00:00:00-04:00,289.060034,289.447329,280.897356,283.250824,10382100,0.0,0.0


In [18]:
# Performance comparison: Sequential vs Concurrent processing
print("=== Performance Comparison ===")

# Test with more tickers to see the performance difference
test_tickers = ["AAPL", "MSFT", "GOOGL", "AMZN", "TSLA", "META", "NFLX", "NVDA"]

print(f"Testing with {len(test_tickers)} tickers: {', '.join(test_tickers)}")

# Sequential processing (max_concurrent=1)
print("\n1. Sequential processing:")
start = time.time()
sequential_results = await fas.get_multiple_fundamentals(
    tickers=test_tickers, field_set="basic_info", max_concurrent=1
)
sequential_time = time.time() - start
print(f"   Time: {sequential_time:.2f} seconds")

# Concurrent processing (max_concurrent=5)
print("\n2. Concurrent processing:")
start = time.time()
concurrent_results = await fas.get_multiple_fundamentals(
    tickers=test_tickers, field_set="basic_info", max_concurrent=5
)
concurrent_time = time.time() - start
print(f"   Time: {concurrent_time:.2f} seconds")

# Calculate speedup
if concurrent_time > 0:
    speedup = sequential_time / concurrent_time
    print(f"\n🚀 Speedup: {speedup:.1f}x faster with concurrency!")
    print(f"   Time saved: {sequential_time - concurrent_time:.1f} seconds")

# Show some results
print(f"\nSuccessfully fetched data for {len(concurrent_results)} tickers:")
for result in concurrent_results[:3]:  # Show first 3
    if "error" not in result:
        ticker = result["ticker"]
        market_cap = result["data"].get("marketCap", "N/A")
        print(f"  {ticker}: Market Cap = {market_cap}")
print("  ...")

=== Performance Comparison ===
Testing with 8 tickers: AAPL, MSFT, GOOGL, AMZN, TSLA, META, NFLX, NVDA

1. Sequential processing:
   Time: 1.80 seconds

2. Concurrent processing:
   Time: 0.33 seconds

🚀 Speedup: 5.5x faster with concurrency!
   Time saved: 1.5 seconds

Successfully fetched data for 8 tickers:
  AAPL: Market Cap = 3473692426240
  MSFT: Market Cap = 3790173372416
  GOOGL: Market Cap = 2915389669376
  ...


In [19]:
# Fetch valuation metrics for the test tickers sequentially (no concurrency)
# This uses the "valuation_metrics" field set, which is available in all_field_sets

sequential_results = await fas.get_multiple_fundamentals(
    tickers=test_tickers,
    field_set="valuation_metrics",
)

# Display results in a user-friendly format
for result in sequential_results:
    ticker = result["ticker"]
    data = result.get("data", {})
    if "error" in result:
        print(f"{ticker}: Error - {result['error']}")
    else:
        print(f"\n{ticker} Valuation Metrics:")
        for field, value in data.items():
            print(f"  {field}: {value}")


AAPL Valuation Metrics:
  enterpriseValue: 3520016154624
  priceToBook: 52.825546
  priceToSalesTrailing12Months: 8.50093
  trailingPE: 35.465153
  forwardPE: 28.167269
  pegRatio: None
  enterpriseToEbitda: 24.842

MSFT Valuation Metrics:
  enterpriseValue: 3807790497792
  priceToBook: 11.035841
  priceToSalesTrailing12Months: 13.453499
  trailingPE: 37.30066
  forwardPE: 34.107025
  pegRatio: None
  enterpriseToEbitda: 24.327

GOOGL Valuation Metrics:
  enterpriseValue: 2858755293184
  priceToBook: 8.031218
  priceToSalesTrailing12Months: 7.849751
  trailingPE: 25.644302
  forwardPE: 26.875
  pegRatio: None
  enterpriseToEbitda: 20.298

AMZN Valuation Metrics:
  enterpriseValue: 2499589701632
  priceToBook: 7.2865763
  priceToSalesTrailing12Months: 3.6314309
  trailingPE: 34.726025
  forwardPE: 37.09756
  pegRatio: None
  enterpriseToEbitda: 18.677

TSLA Valuation Metrics:
  enterpriseValue: 1254194282496
  priceToBook: 16.51057
  priceToSalesTrailing12Months: 13.773562
  trailingPE

In [20]:
# Recreate analyzer to pick up config changes
print("=== Recreating analyzer with updated config ===")
fas = await FundamentalStockAnalyzer.create()

# Check available field sets
print(f"Available field sets: {fas.get_available_field_sets()}")

# Check if our new percentage_changes field group is available
try:
    percentage_fields = fas.get_fields("percentage_changes")
    print(f"Percentage change fields: {percentage_fields}")
except ValueError as e:
    print(f"Error: {e}")

# Check if price_performance field set is available
try:
    price_performance_fields = fas.get_fields("price_performance")
    print(f"Price performance fields: {price_performance_fields}")
except ValueError as e:
    print(f"Error: {e}")

=== Recreating analyzer with updated config ===
Available field sets: ['quick_overview', 'value_analysis', 'fundamental_analysis', 'price_performance', 'complete', 'basic_info', 'valuation_metrics', 'profitability_metrics', 'cash_flow_metrics', 'growth_metrics', 'percentage_changes', 'financial_health', 'dividend_info', 'analyst_data', 'earnings_data', 'all']
Percentage change fields: ['annualChangePercent', 'sixMonthChangePercent', 'threeMonthChangePercent', 'oneMonthChangePercent', 'oneWeekChangePercent']
Price performance fields: ['currentPrice', 'annualChangePercent', 'sixMonthChangePercent', 'threeMonthChangePercent', 'oneMonthChangePercent', 'oneWeekChangePercent']


In [21]:
# Test the percentage change calculations
print("=== Testing Percentage Change Calculations ===")

# Test with AAPL
print("Testing AAPL percentage changes:")
result = await fas.get_fundamentals("AAPL", field_set="percentage_changes")
if "error" not in result:
    for field, value in result["data"].items():
        print(f"  {field}: {value}%")
else:
    print(f"Error: {result['error']}")

print("\n" + "=" * 50)

# Test with price_performance field set for MSFT
print("Testing MSFT price performance:")
result = await fas.get_fundamentals("MSFT", field_set="price_performance")
if "error" not in result:
    for field, value in result["data"].items():
        if field == "currentPrice":
            print(f"  {field}: ${value}")
        else:
            print(f"  {field}: {value}%")
else:
    print(f"Error: {result['error']}")

print("\n" + "=" * 50)

# Test multiple tickers with percentage changes
print("Testing multiple tickers with percentage changes:")
sample_tickers = ["AAPL", "GOOGL"]
results = await fas.get_multiple_fundamentals(
    tickers=sample_tickers, field_set="percentage_changes", max_concurrent=2
)

for result in results:
    ticker = result["ticker"]
    if "error" not in result:
        print(f"\n{ticker}:")
        annual = result["data"].get("annualChangePercent")
        one_week = result["data"].get("oneWeekChangePercent")
        print(f"  Annual Change: {annual}%")
        print(f"  One Week Change: {one_week}%")
    else:
        print(f"{ticker}: Error - {result['error']}")

=== Testing Percentage Change Calculations ===
Testing AAPL percentage changes:
  annualChangePercent: 5.69%
  sixMonthChangePercent: 9.91%
  threeMonthChangePercent: 19.29%
  oneMonthChangePercent: 1.19%
  oneWeekChangePercent: -2.34%

Testing MSFT price performance:
  currentPrice: $509.9
  annualChangePercent: 19.31%
  sixMonthChangePercent: 31.69%
  threeMonthChangePercent: 7.53%
  oneMonthChangePercent: -1.81%
  oneWeekChangePercent: 3.01%

Testing multiple tickers with percentage changes:

AAPL:
  Annual Change: 5.69%
  One Week Change: -2.34%

GOOGL:
  Annual Change: 53.81%
  One Week Change: 2.56%


In [22]:
# Final integration test - combining traditional fundamentals with percentage changes
print("=== Integration Test: Traditional + Percentage Changes ===")

# Test mixing basic info with percentage changes
result = await fas.get_fundamentals("TSLA", field_set="basic_info")
print("TSLA basic info (without percentage changes):")
for field, value in result["data"].items():
    print(f"  {field}: {value}")

print("\n" + "-" * 40)

# Now test basic_info + specific percentage fields by using "all" field set
result = await fas.get_fundamentals("TSLA", field_set="all")
print("TSLA sample of all fields (showing some basics + percentage changes):")
data = result["data"]
sample_fields = [
    "shortName",
    "currentPrice",
    "marketCap",
    "annualChangePercent",
    "sixMonthChangePercent",
    "oneWeekChangePercent",
]

for field in sample_fields:
    value = data.get(field)
    if field in [
        "annualChangePercent",
        "sixMonthChangePercent",
        "oneWeekChangePercent",
    ]:
        print(f"  {field}: {value}%")
    elif field == "currentPrice":
        print(f"  {field}: ${value}")
    else:
        print(f"  {field}: {value}")

print(
    f"\n✅ Successfully integrated {len([f for f in data.keys() if 'ChangePercent' in f])} percentage change fields!"
)
print("✅ All percentage change calculations completed successfully!")
print("✅ Configuration system updated with new field groups!")
print("✅ Async implementation maintains performance!")

=== Integration Test: Traditional + Percentage Changes ===
TSLA basic info (without percentage changes):
  shortName: Tesla, Inc.
  sector: Consumer Cyclical
  industry: Auto Manufacturers
  marketCap: 1277084696576
  currentPrice: 395.94
  beta: 2.065

----------------------------------------
TSLA sample of all fields (showing some basics + percentage changes):
  shortName: Tesla, Inc.
  currentPrice: $395.94
  marketCap: 1277084696576
  annualChangePercent: 71.93%
  sixMonthChangePercent: 58.39%
  oneWeekChangePercent: 12.85%

✅ Successfully integrated 5 percentage change fields!
✅ All percentage change calculations completed successfully!
✅ Configuration system updated with new field groups!
✅ Async implementation maintains performance!
